In [1]:
import os
os.chdir('../')
%pwd

'c:\\Users\\anuru\\Desktop\\MyPro\\Text-Summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [6]:
class DataIgestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: {headers}")
        else:
            logger.infor(f"File already exists of size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)
    
    

In [7]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIgestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2025-12-19 10:26:42,265: INFO: common: Created directory at: artifacts ]
[ 2025-12-19 10:26:42,267: INFO: common: Created directory at: artifacts/data_ingestion ]
[ 2025-12-19 10:26:46,406: INFO: 3216128275: artifacts\data_ingestion\data.zip downloaded! with following info: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2E52:1F6515:9F186:147AC0:6944DB08
Accept-Ranges: bytes
Date: Fri, 19 Dec 2025 04:56:45 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830030-SIN
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1766120205.949585,VS0,VE481
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-or